In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [5]:
word_to_ix={"hello":0,"world":1}
embeds=nn.Embedding(2,5)
lookup_tensor=torch.tensor([word_to_ix["hello"]],dtype=torch.long)
hello_embed=embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [6]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):

    def __init__(self,vocab_size,embedding_dim,context_size):
        super(NGramLanguageModeler,self).__init__()
        self.embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.linear1=nn.Linear(context_size*embedding_dim,128)
        self.linear2=nn.Linear(128,vocab_size)

    def forward(self,inputs):
        embeds=self.embeddings(inputs).view((1,-1))#view ==reshape?
        out=F.relu(self.linear1(embeds))
        out=self.linear2(out)
        log_probs=F.log_softmax(out,dim=1)
        return log_probs

losses=[]
loss_function=nn.NLLLoss()
model=NGramLanguageModeler(len(vocab),EMBEDDING_DIM,CONTEXT_SIZE)
optimizer=optim.SGD(model.parameters(),lr=0.001)

for epoch in range(10):
    total_loss=0
    for context,target in ngrams:
        context_idxs=torch.tensor([word_to_ix[w] for w in context],dtype=torch.long)

        model.zero_grad()

        log_probs=model(context_idxs)

        loss=loss_function(log_probs,torch.tensor([word_to_ix[target]],dtype=torch.long))

        loss.backward()
        optimizer.step()

        total_loss+=loss.item()
    losses.append(total_loss)

print(losses)

print(model.embeddings.weight[word_to_ix['beauty']])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
[525.2006106376648, 522.4754686355591, 519.7737712860107, 517.0938696861267, 514.4331469535828, 511.791307926178, 509.16636896133423, 506.5581052303314, 503.965904712677, 501.38914132118225]
tensor([ 1.3839, -0.8137, -0.9272,  1.1122,  0.6131,  0.1945, -2.5820,  0.8558,
        -2.1038, -0.6192], grad_fn=<SelectBackward0>)


# Exercise

In [29]:
CONTEXT_SIZE = 2 # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])
print(word_to_ix.keys())
print(vocab_size)

class CBOW(nn.Module):

    def __init__(self,vocab_size,embedding_dim,context_size):
        super(CBOW,self).__init__()
        self.embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.Linear1=nn.Linear(context_size*embedding_dim,128)
        self.Linear2=nn.Linear(128,vocab_size)

    def forward(self, inputs):
        embeds=self.embeddings(inputs).view(1,-1)
        out=F.relu(self.Linear1(embeds))
        out=self.Linear2(out)
        log_probs=F.log_softmax(out,dim=1)
        return log_probs

# Create your model and train. Here are some functions to help you make
# the data ready for use by your module.

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)  # example


losses=[]
loss_function=nn.NLLLoss()
model=CBOW(vocab_size,EMBEDDING_DIM,CONTEXT_SIZE*2)
optimizer=optim.SGD(model.parameters(),lr=0.001)

for epoch in range(10):
    total_loss=0
    for context,target in data:
        context_idxs=make_context_vector(context,word_to_ix)
        model.zero_grad()

        log_probs=model(context_idxs)

        loss=loss_function(log_probs,torch.tensor([word_to_ix[target]],dtype=torch.long))

        loss.backward()
        optimizer.step()

        total_loss+=loss.item()
    losses.append(total_loss)

print(losses)

print(model.embeddings.weight[word_to_ix['pattern']])


[(['are', 'We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]
dict_keys(['The', 'As', 'Computational', 'inhabit', 'processes', 'beings', 'our', 'are', 'called', 'process.', 'conjure', 'evolution', 'direct', 'we', 'spirits', 'of', 'create', 'directed', 'things', 'computational', 'manipulate', 'pattern', 'abstract', 'In', 'programs', 'the', 'that', 'study', 'evolve,', 'spells.', 'to', 'a', 'about', 'process', 'data.', 'processes.', 'with', 'rules', 'computer', 'We', 'idea', 'computers.', 'is', 'People', 'by', 'other', 'they', 'program.', 'effect,'])
49
[228.643328666687, 227.18488883972168, 225.73658299446106, 224.29793000221252, 222.8684265613556, 221.4464988708496, 220.03250312805176, 218.62476420402527, 217.224102973938, 215.82932925224304]
tensor([ 1.1214, -0.2585, -0.7867,  1.2850,  1.1023,  1.3475, -0.9912,  0.4632,
        -0.5885, -0.3254